In [7]:
import os
os.chdir(os.getcwd().replace('/Code', ''))

In [8]:
from pathlib import Path

In [9]:
import pandas as pd
import gzip
import pickle as pkl
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import numpy as np
from scipy.sparse import coo_matrix

from sklearn import decomposition, preprocessing
import seaborn as sns
from sklearn.manifold import TSNE

In [10]:
import gensim, spacy
import gensim.corpora as corpora
from nltk.corpus import stopwords

import pandas as pd
import re
from tqdm import tqdm
import time

In [11]:
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
# import matplotlib.pyplot as plt
# %matplotlib inline

## Setup nlp for spacy
nlp = spacy.load("en_core_web_sm")

# Load NLTK stopwords
stop_words = stopwords.words('english')
# Add some extra words in it if required
stop_words.extend(['from', 'subject', 'use','pron'])

In [12]:
with gzip.open('Matrices/CVs_SW_old.pkl.gz', 'rb') as f:
        CVs = pkl.load(f)
CVs.shape

(8083, 4)

## LDA ANALYSE

In [13]:
data_words=CVs["vocab_cv_list"]

In [14]:
data = data_words.values.tolist()

In [15]:
data_words_clean = list(data)

In [16]:
dictionary = corpora.Dictionary(data_words_clean)


In [17]:
corpus = [dictionary.doc2bow(text) for text in data_words_clean]

In [18]:
ldamodel = gensim.models.ldamodel.LdaModel.load('LDA_NYTBIS')

In [19]:
ldamodel.print_topics(-1)

[(0,
  '0.028*"sql" + 0.028*"server" + 0.018*"environment" + 0.017*"production" + 0.017*"database" + 0.015*"unix" + 0.013*"administration" + 0.012*"support" + 0.010*"shell" + 0.009*"dba"'),
 (1,
  '0.016*"manager" + 0.013*"customer" + 0.009*"digital" + 0.008*"support" + 0.008*"marketing" + 0.008*"service" + 0.007*"sale" + 0.007*"production" + 0.007*"communication" + 0.007*"video"'),
 (2,
  '0.022*"server" + 0.021*"windows" + 0.021*"network" + 0.014*"security" + 0.012*"infrastructure" + 0.011*"deployment" + 0.011*"administration" + 0.010*"cisco" + 0.010*"engineer" + 0.009*"environment"'),
 (3,
  '0.019*"functional" + 0.012*"monitoring" + 0.011*"manager" + 0.010*"specification" + 0.010*"test" + 0.008*"tool" + 0.008*"support" + 0.007*"user" + 0.007*"accounting" + 0.007*"plan"'),
 (4,
  '0.012*"risk" + 0.011*"market" + 0.011*"financial" + 0.011*"office" + 0.009*"analyst" + 0.008*"finance" + 0.007*"support" + 0.007*"customer" + 0.007*"product" + 0.007*"bank"'),
 (5,
  '0.038*"java" + 0.014*

In [20]:
def label_topic(num):
    if num==0:
        label="DBA "
    elif num==1:
        label="Media"
    elif num==2:
        label="Sys ops"
    elif num==3:
        label="MOA"
    elif num==4:
        label="Analyste risque"
    elif num==5:
        label="dev Back-End"
    elif num==6:
        label="dev Full Stack"
    elif num==7:
        label="dev Front-End"
    elif num==8:
        label="Data Ing"
    return label
        

In [21]:
import gzip
import pickle as pkl
import pandas as pd
import langdetect
from textblob import TextBlob
from multiprocessing import Pool
from functools import reduce
from collections import Counter
from googletrans import Translator

In [22]:
def translate_cv(text):
        try :
            blob = TextBlob(text )
            return str(blob.translate(from_lang='fr', to='en'))
        except:
            return text
        
    
    
    

In [60]:
def verif_egalite(tab):   
    eg=False
    for i in range(len(tab)-1):
        if tab['prob'][i]==tab['prob'][i+1]:
            eg=True
    return eg

In [68]:
def predict_doc(text):
    global translate_cv
    global label_topic
    global verif_egalite
    
    data=translate_cv(text)
    # Remove single quotes
    data = re.sub("\'", "", data)

    ### Lemmatization
    data_lemma = []
    lis = []
    doc = nlp(data)
    for token in doc:
        lis.append(token.lemma_)
    data_lemma.append(' '.join(lis))

    ### Tokenization and gensim stopword removal

    # You can look for all gensim stopwords by running -> 'gensim.parsing.preprocessing.STOPWORDS'

    # Function to tokenize
    # Also remove words whose length less than 3 (you can chang it)
    def tokenization_with_gen_stop(text):
        result=[]
        for token in gensim.utils.simple_preprocess(text) :
            if token not in gensim.parsing.preprocessing.STOPWORDS:
                result.append(token)

        return result

    ## Apply tokenization function
    data_words = []
    for txt in tqdm(data_lemma):
        data_words.append(tokenization_with_gen_stop(txt))

    ### NLTK Stopword removal (extra stopwords)

    data_words_clean_new = []
    for word in tqdm(data_words):
        for w in word:
            if w not in stop_words:
                data_words_clean_new.append(w)
    ##############################################################################################
    corpus_new = dictionary.doc2bow(data_words_clean_new)
    liste=ldamodel.get_document_topics(corpus_new)
    #table probas
    tab=pd.DataFrame(liste)
    tab.columns=['topic','prob']
    
    
    if verif_egalite(tab)==False:
        #num topic
        df=tab.loc[tab['prob']==max(tab['prob'])]
        num=df.iloc[0]['topic']
        #keys
        topic_prob = ldamodel.get_topic_terms(topicid=int(num))
        topic_doc=[]
        for topic in topic_prob:
            topic_doc.append(dictionary[topic[0]])
        #nom topic
        nom_topic=label_topic(num)
    else:
        tab=pd.DataFrame()
        topic_doc=[]
        num=""
        nom_topic="ce CV ne correspond à aucun topic"
    
        
    
    return tab,topic_doc,num,nom_topic
    
    
    

In [75]:
text="big data r hadoop spark"

In [76]:
tab,topic_doc,num,nom_topic=predict_doc(text)

100%|██████████| 1/1 [00:00<00:00, 733.91it/s]


In [80]:
print(num)
print(nom_topic)
print(topic_doc)
print(tab)


8.0
Data Ing
['data', 'datum', 'big', 'r', 'python', 'science', 'university', 'design', 'computer', 'engineer']
   topic      prob
0      0  0.022224
1      1  0.022223
2      2  0.022223
3      3  0.022223
4      4  0.022223
5      5  0.022223
6      6  0.022223
7      7  0.022223
8      8  0.822217
